## 提取文本的时间

## 使用ltp库
[doc](https://ltp.readthedocs.io/zh_CN/latest/quickstart.html)
[github](https://github.com/HIT-SCIR/ltp)

In [3]:
from ltp import LTP
ltp = LTP() # 默认加载 Small 模型

### 分句

In [19]:
sents = ltp.sent_split(["他叫汤姆去拿外衣。", "汤姆生病了.。他去了医院。"])

In [20]:
sents

['他叫汤姆去拿外衣。', '汤姆生病了.', '。', '他去了医院。']

### 分词

In [5]:
segment, _ = ltp.seg(["他叫汤姆去拿外衣。"])
print(segment, )

[['他', '叫', '汤姆', '去', '拿', '外衣', '。']]


###  ner
```
{
    "Nh": "person",
    "Ns": "loc",
    "Ni": "org"
}
```

In [7]:
seg, hidden = ltp.seg(["在美国主导下，美英澳三国于2021年9月15日宣布建立新的三边安全伙伴关系，美英将支持澳海军建立核潜艇部队，澳将与美英合作在澳建造核潜艇。澳方随即宣布，撕毁与法国海军集团签订的数百亿美元潜艇大单。"])
ner = ltp.ner(hidden)
# [['他', '叫', '汤姆', '去', '拿', '外衣', '。']]
# [[('Nh', 2, 2)]]

tag_dict = {
    "Nh": "person",
    "Ns": "loc",
    "Ni": "org"
}

for tag, start, end in ner[0]:
    print(tag_dict[tag],":", start, "".join(seg[0][start:end + 1]))

loc : 1 美国
loc : 5 美
loc : 6 英
loc : 7 澳
loc : 23 美
loc : 24 英
loc : 27 澳
loc : 33 澳
loc : 36 美英
loc : 39 澳
org : 49 法国


In [5]:
ner

[[('Ns', 1, 1),
  ('Ns', 5, 5),
  ('Ns', 6, 6),
  ('Ns', 7, 7),
  ('Ns', 23, 23),
  ('Ns', 24, 24),
  ('Ns', 27, 27),
  ('Ns', 33, 33),
  ('Ns', 36, 36),
  ('Ns', 39, 39),
  ('Ni', 49, 49)]]

### 语义角色标注

In [8]:
sent = "在美国主导下，美英澳三国于2021年9月15日宣布建立新的三边安全伙伴关系，美英将支持澳海军建立核潜艇部队，澳将与美英合作在澳建造核潜艇。澳方随即宣布，撕毁与法国海军集团签订的数百亿美元潜艇大单。"

In [9]:
sents = ltp.sent_split([sent])

In [10]:
print(sents)

['在美国主导下，美英澳三国于2021年9月15日宣布建立新的三边安全伙伴关系，美英将支持澳海军建立核潜艇部队，澳将与美英合作在澳建造核潜艇。', '澳方随即宣布，撕毁与法国海军集团签订的数百亿美元潜艇大单。']


In [11]:
segs, hidden = ltp.seg(sents)
srls = ltp.srl(hidden, keep_empty=False)

In [12]:
print(segs)

[['在', '美国', '主导', '下', '，', '美', '英', '澳', '三', '国', '于', '2021年', '9月', '15日', '宣布', '建立', '新', '的', '三边', '安全', '伙伴', '关系', '，', '美', '英', '将', '支持', '澳', '海军', '建立', '核潜艇', '部队', '，', '澳', '将', '与', '美英', '合作', '在', '澳', '建造', '核潜艇', '。'], ['澳方', '随即', '宣布', '，', '撕毁', '与', '法国', '海军', '集团', '签订', '的', '数百亿', '美元', '潜艇', '大单', '。']]


In [13]:
print(sent)
print("*"*60)
for seg, srl in zip(segs, srls):
    for token_srl in srl:
        role = seg[token_srl[0]]
        args = token_srl[1]
        
        for arg in args:
            print(arg[0], "".join(seg[arg[1]: arg[2]+1]))
        print("*"*60)

在美国主导下，美英澳三国于2021年9月15日宣布建立新的三边安全伙伴关系，美英将支持澳海军建立核潜艇部队，澳将与美英合作在澳建造核潜艇。澳方随即宣布，撕毁与法国海军集团签订的数百亿美元潜艇大单。
************************************************************
A0 美国
************************************************************
ARGM-MNR 在美国主导下
A0 美英澳三国
ARGM-TMP 于2021年9月15日
A1 建立新的三边安全伙伴关系
************************************************************
A1 新的三边安全伙伴关系
************************************************************
A0 美英
ARGM-ADV 将
A1 澳海军建立核潜艇部队
************************************************************
A0 澳海军
A1 核潜艇部队
************************************************************
A0 澳
ARGM-ADV 将
A0-CRD 与美英
************************************************************
A0 澳
ARGM-LOC 在澳
A1 核潜艇
************************************************************
A0 澳方
ARGM-ADV 随即
A1 撕毁与法国海军集团签订的数百亿美元潜艇大单
************************************************************
A1 与法国海军集团签订的数百亿美元潜艇大单
************************************************************
A0 与法国海军集团
A1 潜艇大单
**************************

### 封装为函数
```
function: srl()
input: text
output: {"text": text, "srl_list":[
                [[type, start, end, text], [type, start, end, text]],
            ]
        }
```

In [5]:
from ltp import LTP
ltp = LTP() # 默认加载 Small 模型

def srl_process(text, is_print = False):
    res = {"text": "", "srl_list":[]}
    sents = ltp.sent_split([text])
    segs, hidden = ltp.seg(sents)
    srls = ltp.srl(hidden, keep_empty=False)
    cur_text = ""
    cur_len = 0
    for seg, srl in zip(segs, srls):
        tmp = []
        for token_srl in srl:
            role = seg[token_srl[0]]
            args = token_srl[1]
            args_type = [it[0] for it in args]
            if "ARGM-TMP" in args_type:
                for arg in args:
                    if is_print:
                        print(arg[0], "".join(seg[arg[1]: arg[2]+1]))
                    start = cur_len+len("".join(seg[:arg[1]]))
                    end = cur_len+len("".join(seg[:arg[2]+1]))
                    _type = arg[0]
                    if _type == "ARGM-TMP":
                        _type = "temporal"
                    tmp.append([_type, start, end, "".join(seg[arg[1]: arg[2]+1])])
                if is_print: 
                    print("*"*60)
        cur_text += "".join(seg)
        cur_len += len("".join(seg))
        if len(tmp) > 0:
            res["srl_list"].append(tmp)
    res["text"] = cur_text
    return res
        
    

In [6]:
res = srl_process(
    "美英澳三国9月15日宣布组建名为“AUKUS（奥库斯）”的新安全联盟。作为该联盟的先行合作示范项目，美英将向澳提供核潜艇技术，助其组建核潜艇编队。此举意味着澳单方面撕毁与法国660亿美元（1美元约合6.65元人民币）的12艘常规潜艇采购协议。美英澳围绕核潜艇展开防务合作，不但进一步反映出西方内部矛盾与裂痕，更暴露相关国家在核不扩散问题上奉行的“双重标准”，引发的外溢效应或加剧地区国家间军备竞赛，冲击地区和平。从目前各方披露的信息看，美英将在新安全联盟框架下，向澳研制核潜艇提供必要技术支撑，澳将建造至少8艘核潜艇，总支出将超过659亿美元。为应对未来竞争，美将助澳发展核潜艇作为深化两国防务合作，锐化亚太军力布局的重要一环。美承诺出面协调澳潜艇的核材料供应。鉴于核技术的高度敏感性，美此前仅与英国有过类似合作，这体现出美对澳在亚太扮演重要军事角色的“重视”。据澳媒报道，为让澳方提前熟悉核潜艇的操作规程，美军将于近期派遣数艘弗吉尼亚级潜艇进驻澳斯特林海军基地。同时，美将向澳大利亚轮换部署所有类型美军战机，与澳共同打造为舰艇和战机提供后勤支持的联合作战能力。与美相比，英国拥有在自身潜艇上整合美技术和装备的能力与经验，这成为其为澳潜艇提供技术保障的主要原因。据悉，英国罗尔斯·罗伊斯公司将为澳潜艇提供核反应堆。此外，英将核潜艇项目视为“脱欧”后加速全球战略调整的重要契机，为未来在亚太地区保持长期军事存在奠定合作基础。在澳看来，核潜艇是跻身世界一流海军的标志。一旦协议落地，澳有望成为全球第7个装备核潜艇的国家。澳欲借此实现跨区域军力投送能力，并在短时间内向邻近海域派遣作战单元，扭转长期以来海军发展的颓势。此外，据澳媒16日消息称，为加强澳军事防御，澳将首次获得美国“战斧”巡航导弹。美英澳此番达成核潜艇建造协议，法国的回应最为激烈。法国外长和防长在15日发表的联合声明中，一方面指责美方的决定迫使法放弃与澳签署的潜艇采购协议，并将其排挤出与澳方的结构性伙伴关系，实属粗暴且不可预见的“背后捅刀”行为；另一方面批驳澳方的举动有违两国合作精神，不可接受。接着，法国取消与美国共同庆祝纪念“弗吉尼亚大海战240周年”活动。17日，法国总统马克龙下令，召回驻美澳两国大使，将就澳单方面撕毁协议一事进行磋商。同为“五眼联盟”成员的新西兰，对澳方发展核潜艇的举动并不买账。新西兰总理阿德恩表示，该国将继续执行始于1985年的核动力舰艇禁令，决不允许澳核潜艇驶入新西兰附近海域。在澳国内，绿党党首亚当·班特表示不认可莫里森政府的选择，并将核潜艇在当前海上安全环境中的处境比作“浮动的切尔诺贝利”。而澳政府一味追随美国的军事战略取向，将把自身置于地区冲突升级的“最前线”，并不符合澳“中等国家”的定位。此外，考虑到核潜艇本身设计建造的长周期属性，以及围绕相关敏感技术展开的漫长谈判进程，甚至连莫里森本人也不得不承认，澳核潜艇恐将推迟至2040年服役。欧盟委员会主席冯德莱恩近期发表“盟情咨文”表示，“阻碍欧盟在军事领域合作的不仅是能力不足，更是政治意愿的缺失”。此次事件后，作为欧盟一体化主要推动者，法国已表现出强化欧洲“战略自主”的意愿。明年恰逢法国主办欧盟防务峰会，法方势必借此进一步凝聚各方意志，为在缺少北约参与的情况下实施军事干预行动做足准备。美英作为拥核国家，一方面对伊朗等国核计划施以严格限制和打压，另一方面却扶持澳发展军用核技术，这种明目张胆",
    False
)

# print(res['text'])
for _srl in res["srl_list"]:
    for one_srl in _srl:
#         print(_srl)
        print(res['text'][one_srl[1]:one_srl[2]] == one_srl[3])
        print(one_srl)
#         print(_srl)
    print("*"*60)

True
['A0', 0, 5, '美英澳三国']
True
['temporal', 5, 10, '9月15日']
True
['A1', 12, 34, '组建名为“AUKUS（奥库斯）”的新安全联盟']
************************************************************
True
['temporal', 207, 209, '目前']
True
['A0', 209, 211, '各方']
True
['A1', 214, 216, '信息']
************************************************************
True
['ARGM-MNR', 381, 386, '据澳媒报道']
True
['ARGM-PRP', 387, 403, '为让澳方提前熟悉核潜艇的操作规程']
True
['A0', 404, 406, '美军']
True
['ARGM-ADV', 406, 407, '将']
True
['temporal', 407, 410, '于近期']
True
['A1', 412, 421, '数艘弗吉尼亚级潜艇']
True
['A2', 421, 431, '进驻澳斯特林海军基地']
************************************************************
True
['temporal', 570, 574, '脱欧”后']
True
['A1', 576, 582, '全球战略调整']
True
['ARGM-MNR', 585, 587, '契机']
True
['temporal', 589, 591, '未来']
True
['ARGM-LOC', 591, 596, '在亚太地区']
True
['A1', 598, 604, '长期军事存在']
************************************************************
True
['A0', 658, 660, '澳欲']
True
['temporal', 675, 680, '在短时间内']
True
['ARGM-DIR', 680, 685, '向邻近海域']
